```{=latex}
\usepackage{hyperref}
\usepackage{graphicx}
\usepackage{listings}
\usepackage{textcomp}
\usepackage{fancyvrb}

\newcommand{\passthrough}[1]{\lstset{mathescape=false}#1\lstset{mathescape=true}}
\newcommand{\tightlist}{}
```

```{=latex}
\title{End-to-End Kubernetes}
\author{Moshe Zadka -- https://cobordism.com}
\date{}

\begin{document}
\begin{titlepage}
\maketitle
\end{titlepage}

\frame{\titlepage}
```

```{=latex}
\begin{frame}
\frametitle{Acknowledgement of Country}

Belmont (in San Francisco Bay Area Peninsula)

Ancestral homeland of the Ramaytush Ohlone

\end{frame}
```

## Introduction

### Software as a pipeline

### Software as a leaky pipeline

### Examples as caricatures

### Goal: fixing leaks

### Talk goal: Mental model

### Example application

```python
# e2e_k8s/demo.py
from pyramid import response, config, view
@view.view_config(route_name='add')
def add(request):
    x, y = (int(request.matchdict[c]) for c in "xy")
    return response.Response(str(x+y))
with config.Configurator() as _cfg:
    _cfg.add_route("add", "/add/{x}/{y}")
    _cfg.scan("e2e_k8s")
    application = _cfg.make_wsgi_app()
```

## Introduction

### What is kubernetes?

### Why is kubernetes popular?

### What is DevOps?

### Why is DevOps good?

## Scope

### Multi-cluster (beyond scope)

### Database migration (beyond scope)

### Stateful applications (beyond scope)

### Advanced deployment (beyond scopre)

## Software Development Lifecycle

### What is an edit debug cycle?

### Why edit-debug cycle latency matters

## Kubernetes Application Architecture

### Containers

### Kubernetes

### Pods

### Deployments

## Environments

### What is an environment?

### Cross-talk

### Regional Separation

### Maturity Separation

## Modern SDLC Caricature

### Develop

### Review

### Merge

### Deploy

### No "test" stage

## Kubernetes across environments

### Why

### Challenges

### Lima

### Minikube

### Cloud

## Customizing Kubernetes

### Ad-hoc

### Template

### Kustomize

### Helm

### Operators

### 

## Kubernetes Pods

### What are pods?

### Container namespaces

### Interpod access

## Kubernetes routing

### Service

### Headless service

### Pod

### StatefulSet

## Updating Containers

### SSH to Pod

### Modifying files

### Synchornizing files

### Auto-restarting

## Putting it all together

### Build container

### Configure Pod

### Sync files

### Update container

### See results

## Summary

### K8s as a universal platform

### K8s end-to-end

```{=latex}
\end{document}
```